In [1]:
## Mistral

In [2]:
%pip install --upgrade accelerate
%pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [3]:
!pip install gdown

In [4]:
import gdown

# Replace with your actual Google Drive file ID or the full URL
FILE_ID = '1qzbTiJ2IL5ATZYNMp2DRkHvbFYsnOVAQ'
file_url = f'https://drive.google.com/uc?id={FILE_ID}'   # or use the full URL directly
output = 'filename.zip'  # Replace with the desired output filename

gdown.download(file_url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1qzbTiJ2IL5ATZYNMp2DRkHvbFYsnOVAQ
From (redirected): https://drive.google.com/uc?id=1qzbTiJ2IL5ATZYNMp2DRkHvbFYsnOVAQ&confirm=t&uuid=bba86a78-3c16-484a-a6aa-22f4c1bab9f4
To: /content/filename.zip
100%|██████████| 250M/250M [00:01<00:00, 155MB/s]


'filename.zip'

In [5]:
!unzip /content/filename.zip

Archive:  /content/filename.zip
   creating: datasets/
  inflating: datasets/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/datasets/
  inflating: __MACOSX/datasets/._.DS_Store  
   creating: datasets/beer/
  inflating: datasets/beer/test.txt  
  inflating: datasets/beer/test_label.txt  
  inflating: datasets/beer/train.txt  
   creating: datasets/restaurant/
  inflating: datasets/restaurant/test.txt  
  inflating: datasets/restaurant/test_label.txt  
  inflating: datasets/restaurant/train.txt  


In [6]:
from pathlib import Path
review_domain = 'restaurant'

input_dir = Path(f'/content/datasets/{review_domain}')
data_dir = Path('/content/datasets/')

In [7]:
import pandas as pd

def load_dataset(filename):
    f = open(filename, 'r', encoding='utf-8')
    all_reviews = f.readlines()
    print('Total Reviews: ', len(all_reviews))
    f.close()

    sentences = []
    for i,review in enumerate(all_reviews):
        sentences.append(review.strip('\n'))

    col = 'labels' if 'test_label' in str(filename) else 'text_org'

    df = pd.DataFrame({col:sentences})
    return df

df_label = load_dataset(input_dir/'test_label.txt')
print(df_label.shape)

Total Reviews:  3328
(3328, 1)


In [8]:


all_labels = list(set(df_label['labels'].str.split(' ').sum()))
print(all_labels)
drop_labels = ['Positive', 'Neutral']
all_labels = sorted([label for label in all_labels if label not in drop_labels])

drop_index = pd.Series(False, index=df_label.index)
for label in drop_labels:
    drop_index = drop_index | df_label['labels'].str.contains(label)

print(all_labels)

['Neutral', 'Staff', 'Anecdotes', 'Miscellaneous', 'Price', 'Positive', 'Food', 'Ambience']
['Ambience', 'Anecdotes', 'Food', 'Miscellaneous', 'Price', 'Staff']


In [9]:

import numpy as np
df_label = df_label[~drop_index].copy()
print(df_label.shape)

for label in all_labels:
    df_label[label] = np.where(df_label['labels'].str.contains(label), 1, 0)
print(df_label.shape)

df_label.head()

df = load_dataset(input_dir/'test.txt')
print(df.shape)

(3315, 1)
(3315, 7)
Total Reviews:  3328
(3328, 1)


In [10]:


df = df[~drop_index].copy()
print(df.shape)

df = pd.concat([df, df_label], axis=1)

df.head()

(3315, 1)


,text_org,labels,Ambience,Anecdotes,Food,Miscellaneous,Price,Staff
0,Always a fun place ... the food is deeelish !,Food Ambience,1,0,1,0,0,0
1,The staff is n't the friendliest or most comp...,Staff,0,0,0,0,0,1
2,"Great for groups , great for a date , great f...",Ambience,1,0,0,0,0,0
3,Another great place to take out-of-towners !,Miscellaneous,0,0,0,1,0,0
4,: ),Miscellaneous,0,0,0,1,0,0


In [11]:

df[all_labels].mean()

,0
Ambience,0.113424
Anecdotes,0.129412
Food,0.370136
Miscellaneous,0.276621
Price,0.089894
Staff,0.166817


In [12]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [13]:
# %pip install --upgrade accelerate
## Setup LLM: Mistral 7B

# !pip install transformers==4.20
# %pip install sentencepiece
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
#repo_id = "Open-Orca/Mistral-7B-OpenOrca"
repo_id = 'meta-llama/Meta-Llama-3.1-8B'

from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(repo_id, torch_dtype="auto", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(repo_id, torch_dtype="auto", low_cpu_mem_usage=True)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [14]:
%pip install langchain langchain_core langchain_community kor tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFaceHub, HuggingFacePipeline
import torch
torch.cuda.empty_cache()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 50,
    device=device,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.01,
)

llm = HuggingFacePipeline(pipeline=pipe)

# from transformers.pipelines.pt_utils import KeyDataset
# for out in pipe(KeyDataset(dataset, "text"), batch_size=8, truncation="only_first"):
#     print(out)



In [21]:
import langchain
from typing import List, Union, Optional
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate,SystemMessagePromptTemplate,AIMessagePromptTemplate,HumanMessagePromptTemplate
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, Field, validator
import enum
from kor import extract_from_documents, from_pydantic
from kor.extraction import create_extraction_chain

## Setup Aspect Extractor

class Aspects(enum.Enum):
    staff = "Staff"
    ambience = "Ambience"
    anecdotes = "Anecdotes"
    food = "Food"
    price = 'Price'
    miscellaneous = 'Miscellaneous'

class ReviewABSA(BaseModel):
    aspect: List[Aspects] = Field(
        description=f"The key features that customers are talking about in their {review_domain} reviews.",
        examples=[
            ("The Singapore Mai Fun had NO curry flavor whatsoever.", "Food"),
            ("Go for it !", "Miscellaneous"),
            ("While the staff at this little bistro is very friendly , I have never experienced more incompetency.", "Staff"),
            ("Went on a double date with friend and his girlfriend for a few drinks and appetizers .", "Anecdotes"),
            ("When you enter , you feel like you have entered your mom 's kitchen .", "Ambience"),
            ("Pricey , but worth a try , at least once .", "Price")
        ],
        many=True,
    )

instruction_template = PromptTemplate(
    input_variables=["format_instructions", "type_description"],
    template=(
        "<|im_start|>system\nYour goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.\n\n"
        "{type_description}\n<|im_end|>\n<|im_start|>user\n{format_instructions}\n"
    ),
)

#from langchain.chains import create_extraction_chain
schema, validator = from_pydantic(ReviewABSA)
chain = create_extraction_chain(
    llm, schema, validator=validator,
    encoder_or_encoder_class="json",
    instruction_template=instruction_template
)

print(chain.get_prompts()[0].format_prompt(text="[user input]").to_string())

<|im_start|>system
Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

reviewabsa: { // 
 aspect: Array<"Staff" | "Ambience" | "Anecdotes" | "Food" | "Price" | "Miscellaneous"> // The key features that customers are talking about in their restaurant reviews.
}
```

<|im_end|>
<|im_start|>user
Please output the extracted information in JSON format. Do not output anything except for the extracted information. Do not add any clarifying information. Do not add any fields that are not in the schema. If the text contains attributes that do not appear in the schema, please ignore them. All output must be in JSON format and follow the schema specified above. Wrap the JSON in <json> tags.


Input: The Singapore Mai Fun had NO curry flavor whatsoever.
Output: <json>{"re

In [22]:
%%time
from tqdm import tqdm
reviews = df['text_org'].values.tolist()
sample = 300
reviews_sampled=reviews[0:sample]
responses = []
aspects = []

for text in tqdm(reviews_sampled):
    response = chain.invoke(input=text+'<|im_end|>\n')
    responses.append(response)
    aspects.append(response['data']['reviewabsa']['aspect'] if len(response['data'].keys())>0
                   else ['Miscellaneous'])

all_labels_sampled=all_labels[0:sample]
aspects_sampled=aspects[0:sample]
len(reviews), len(responses), len(aspects)

100%|██████████| 300/300 [09:29<00:00,  1.90s/it]

CPU times: user 9min 28s, sys: 258 ms, total: 9min 29s
Wall time: 9min 29s


(3315, 300, 300)

In [23]:
## Aspect Extraction
df_test_pred = pd.DataFrame(columns=[f'predicted_{label}' for label in all_labels_sampled])
df_test_pred['predicted_labels'] = [' '.join(aspect) for aspect in aspects_sampled]
for label in all_labels_sampled:
    df_test_pred[f'predicted_{label}'] = np.where(df_test_pred['predicted_labels'].str.contains(label), 1, 0)
df_test_pred.index = df.iloc[0:sample].index
print(df_test_pred.shape)

(300, 7)


In [24]:
## Evaluate Results
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
df_final = pd.concat([df.iloc[0:sample], df_test_pred], axis=1)
print(df_final.shape)
df_final = df_final[['text_org', 'labels', 'predicted_labels']+all_labels_sampled+[f'predicted_{label}' for label in all_labels_sampled]]
df_final.head()

interested_labels = ['Ambience', 'Food', 'Price', 'Staff']
for label in interested_labels:
    print('Label: ', label)
    y_true = df_final[label]
    y_pred = df_final[f'predicted_{label}']
    print('Accuracy: ', np.round(accuracy_score(y_true, y_pred),3))
    print('F1 Score: ', np.round(f1_score(y_true, y_pred),3))
    print('Precision: ', np.round(precision_score(y_true, y_pred),3))
    print('Recall: ', np.round(recall_score(y_true, y_pred),3))
    print('***************')

(300, 15)
Label:  Ambience
Accuracy:  0.867
F1 Score:  0.0
Precision:  0.0
Recall:  0.0
***************
Label:  Food
Accuracy:  0.677
F1 Score:  0.0
Precision:  0.0
Recall:  0.0
***************
Label:  Price
Accuracy:  0.94
F1 Score:  0.0
Precision:  0.0
Recall:  0.0
***************
Label:  Staff
Accuracy:  0.79
F1 Score:  0.0
Precision:  0.0
Recall:  0.0
***************


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defin

In [25]:
other_labels = ['Anecdotes', 'Miscellaneous']
for label in other_labels:
    print('Label: ', label)
    y_true = df_final[label]
    y_pred = df_final[f'predicted_{label}']
    print('Accuracy: ', np.round(accuracy_score(y_true, y_pred),3))
    print('F1 Score: ', np.round(f1_score(y_true, y_pred),3))
    print('Precision: ', np.round(precision_score(y_true, y_pred),3))
    print('Recall: ', np.round(recall_score(y_true, y_pred),3))
    print('***************')

Label:  Anecdotes
Accuracy:  0.833
F1 Score:  0.0
Precision:  0.0
Recall:  0.0
***************
Label:  Miscellaneous
Accuracy:  0.267
F1 Score:  0.421
Precision:  0.267
Recall:  1.0
***************


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
df_final

,text_org,labels,predicted_labels,Ambience,Anecdotes,Food,Miscellaneous,Price,Staff,predicted_Ambience,predicted_Anecdotes,predicted_Food,predicted_Miscellaneous,predicted_Price,predicted_Staff
0,Always a fun place ... the food is deeelish !,Food Ambience,Miscellaneous,1,0,1,0,0,0,0,0,0,1,0,0
1,The staff is n't the friendliest or most comp...,Staff,Miscellaneous,0,0,0,0,0,1,0,0,0,1,0,0
2,"Great for groups , great for a date , great f...",Ambience,Miscellaneous,1,0,0,0,0,0,0,0,0,1,0,0
3,Another great place to take out-of-towners !,Miscellaneous,Miscellaneous,0,0,0,1,0,0,0,0,0,1,0,0
4,: ),Miscellaneous,Miscellaneous,0,0,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"The fish was adequate , but inexpertly sliced .",Food,Miscellaneous,0,0,1,0,0,0,0,0,0,1,0,0
296,It is obvious that no one in the restaurant h...,Miscellaneous,Miscellaneous,0,0,0,1,0,0,0,0,0,1,0,0
297,Do n't be fooled by crowds of people .,Miscellaneous,Miscellaneous,0,0,0,1,0,0,0,0,0,1,0,0
298,The service is awful .,Staff,Miscellaneous,0,0,0,0,0,1,0,0,0,1,0,0
